In [1]:
import pandas as pd
import numpy as np
import pickle
from konlpy.tag import Kkma
from tqdm import tqdm
from gensim.models import FastText as ft
from __future__ import print_function
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from nltk.tokenize import RegexpTokenizer
import nltk
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import pad_sequence
from konlpy.tag import Hannanum
from nltk import bigrams, word_tokenize
from nltk.util import ngrams

In [2]:
data = pd.read_pickle('seoul_room_1220.pkl')

In [3]:
data.isnull().sum()

id                     0
lat                    0
lng                    0
info                   0
url                    0
address                0
detail                 0
picture                0
type                   0
deposit                0
rental_fee         12426
maintenance_fee        0
doc                    0
doc_new                0
token_new              0
dtype: int64

In [4]:
data

,id,lat,lng,info,url,address,detail,picture,type,deposit,rental_fee,maintenance_fee,doc,doc_new,token_new
144,34402581,37.446504,127.057650,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,"◆◆전입가,무융자,전세대출가,지하철역3분◆이중창,주차가능 ◆◆전입가,무융자,전세대출...",https://ic.zigbang.com/ic/items/34402581/1.jpg,전세,17500,NaN,8.0,서울시 서초구 신원동 전입가 무융자 전세대출가 지하철역 분 이중창 주차가능 전입가 ...,서울시 서초구 신원동 전입가 무융자 전세대출가 지하철역 분 이중창 주차가능 전입가 ...,"[서울시, 서초구, 신원동, 전입, 무, 융자, 대출, 지하철역, 이중창, 주차, ..."
145,34448303,37.446532,127.057644,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34448303/1.jpg,월세,1000,65.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,"[서울시, 서초구, 신원동, 청계산, 입구, 역다, 권, 오피스텔, 청계산, 입구,..."
159,34521223,37.447032,127.056570,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 서초구 신원동,★신축급 풀옵션 투룸 오피스텔 전세 매물★ # 12월8일 이후부터 투어 가능\r\...,https://ic.zigbang.com/ic/items/34521223/1.jpg,전세,37500,NaN,20.0,서울시 서초구 신원동 신축급 풀옵션 투룸 오피스텔 전세 매물 월 일 이후부터 투어 ...,서울시 서초구 신원동 신축급 풀옵션 투룸 오피스텔 전세 매물 월 일 이후부터 투어 ...,"[서울시, 서초구, 신원동, 신축, 풀, 옵션, 투룸, 오피스텔, 전세, 투어, 가..."
165,34493219,37.446730,127.056288,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34493219/1.jpg,전세,17000,NaN,8.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,"[서울시, 서초구, 신원동, 청계산, 입구, 역다, 권, 오피스텔, 청계산, 입구,..."
188,34498546,37.447738,127.056370,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 사업자용 *청계산입구 초 역세권 신축 오피스텔\r\n...,https://ic.zigbang.com/ic/items/34498546/1.jpg,월세,1000,88.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 사업자용 청계산입구 초 역세권 ...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 사업자용 청계산입구 초 역세권 ...,"[서울시, 서초구, 신원동, 청계산, 입구, 역다, 권, 오피스텔, 사업자, 용, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39941,34512135,37.680000,127.042272,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔신축급풀옵션_방큰원룸✔역세권_도봉역 4분✔즉시입주 1. 오픈형원룸/수려한 도봉산경...,https://ic.zigbang.com/ic/items/34512135/1.jpg,월세,500,37.0,3.0,서울시 도봉구 도봉동 신축급풀옵션 방큰원룸 역세권 도봉역 분 즉시입주 오픈형원룸 수...,서울시 도봉구 도봉동 신축급풀옵션 방큰원룸 역세권 도봉역 분 즉시입주 오픈형원룸 수...,"[서울시, 도봉구, 도봉동, 신축, 풀, 옵션, 방, 크다, 원룸, 역다, 권, 도..."
39942,34522766,37.680244,127.044085,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔풀옵션_깨끗✔역세권_1호선도봉역 4분✔버스전용 20개노선 1. 교통편리 : 역세권...,https://ic.zigbang.com/ic/items/34522766/1.jpg,월세,300,30.0,3.0,서울시 도봉구 도봉동 풀옵션 깨끗 역세권 호선도봉역 분 버스전용 개노선 교통편리 역...,서울시 도봉구 도봉동 풀옵션 깨끗 역세권 호선도봉역 분 버스전용 개노선 교통편리 역...,"[서울시, 도봉구, 도봉동, 풀, 옵션, 깨끗, 역다, 권, 도봉역, 버스, 전용,..."
39943,34411514,37.680953,127.043451,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 도봉구 도봉동,🌈도봉역 2019년식 투룸 안심대출가능 주차가능🌈 📌매물정보\r\n 도봉역5...,https://ic.zigbang.com/ic/items/34411514/1.jpg,전세,20400,NaN,4.0,서울시 도봉구 도봉동 도봉역 년식 투룸 안심대출가능 주차가능 매물정보 도봉역 분 분...,서울시 도봉구 도봉동 도봉역 년식 투룸 안심대출가능 주차가능 매물정보 도봉역 분 분...,"[서울시, 도봉구, 도봉동, 도봉역, 투룸, 안심, 대출, 가능, 주차, 가능, 도..."
39944,34575820,37.680461,127.043434,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,💰HUG 안심전세💰초역세권💰깔끔한 투룸 전세💰 🌈 매물 소개 🌈\r\n\r\n1️⃣...,https://ic.zigbang.com/ic/items/34575820/1.jpg,전세,20000,NaN,4.0,서울시 도봉구 도봉동 안심전세 초역세권 깔끔한 투룸 전세 매물 소개 안심전세 대출 ...,서울시 도봉구 도봉동 안심전세 초역세권 깔끔한 투룸 전세 매물 소개 안심전세 대출 ...,"[서울시, 도봉구, 도봉동, 안심, 전세, 역다, 권, 깔끔하다, 투룸, 전세, 소..."


In [6]:
text= []
for i in data['doc_new']:
    text.append(i)

In [7]:
bigram_list = []
for i in text:
    tokens = word_tokenize(i)
    bigram = list(bigrams(tokens))
    haha = []
    for i in bigram:
        imsi = i[0] + " " + i[1]
        haha.append(imsi)
    bigram_list.append(haha)

In [8]:
len(bigram_list)

30637

In [9]:
data['bigram_list']=bigram_list

In [10]:
data

,id,lat,lng,info,url,address,detail,picture,type,deposit,rental_fee,maintenance_fee,doc,doc_new,token_new,bigram_list
144,34402581,37.446504,127.057650,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,"◆◆전입가,무융자,전세대출가,지하철역3분◆이중창,주차가능 ◆◆전입가,무융자,전세대출...",https://ic.zigbang.com/ic/items/34402581/1.jpg,전세,17500,NaN,8.0,서울시 서초구 신원동 전입가 무융자 전세대출가 지하철역 분 이중창 주차가능 전입가 ...,서울시 서초구 신원동 전입가 무융자 전세대출가 지하철역 분 이중창 주차가능 전입가 ...,"[서울시, 서초구, 신원동, 전입, 무, 융자, 대출, 지하철역, 이중창, 주차, ...","[서울시 서초구, 서초구 신원동, 신원동 전입가, 전입가 무융자, 무융자 전세대출가..."
145,34448303,37.446532,127.057644,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34448303/1.jpg,월세,1000,65.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,"[서울시, 서초구, 신원동, 청계산, 입구, 역다, 권, 오피스텔, 청계산, 입구,...","[서울시 서초구, 서초구 신원동, 신원동 청계산입구, 청계산입구 초, 초 역세권, ..."
159,34521223,37.447032,127.056570,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 서초구 신원동,★신축급 풀옵션 투룸 오피스텔 전세 매물★ # 12월8일 이후부터 투어 가능\r\...,https://ic.zigbang.com/ic/items/34521223/1.jpg,전세,37500,NaN,20.0,서울시 서초구 신원동 신축급 풀옵션 투룸 오피스텔 전세 매물 월 일 이후부터 투어 ...,서울시 서초구 신원동 신축급 풀옵션 투룸 오피스텔 전세 매물 월 일 이후부터 투어 ...,"[서울시, 서초구, 신원동, 신축, 풀, 옵션, 투룸, 오피스텔, 전세, 투어, 가...","[서울시 서초구, 서초구 신원동, 신원동 신축급, 신축급 풀옵션, 풀옵션 투룸, 투..."
165,34493219,37.446730,127.056288,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34493219/1.jpg,전세,17000,NaN,8.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,"[서울시, 서초구, 신원동, 청계산, 입구, 역다, 권, 오피스텔, 청계산, 입구,...","[서울시 서초구, 서초구 신원동, 신원동 청계산입구, 청계산입구 초, 초 역세권, ..."
188,34498546,37.447738,127.056370,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 사업자용 *청계산입구 초 역세권 신축 오피스텔\r\n...,https://ic.zigbang.com/ic/items/34498546/1.jpg,월세,1000,88.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 사업자용 청계산입구 초 역세권 ...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 사업자용 청계산입구 초 역세권 ...,"[서울시, 서초구, 신원동, 청계산, 입구, 역다, 권, 오피스텔, 사업자, 용, ...","[서울시 서초구, 서초구 신원동, 신원동 청계산입구, 청계산입구 초, 초 역세권, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39941,34512135,37.680000,127.042272,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔신축급풀옵션_방큰원룸✔역세권_도봉역 4분✔즉시입주 1. 오픈형원룸/수려한 도봉산경...,https://ic.zigbang.com/ic/items/34512135/1.jpg,월세,500,37.0,3.0,서울시 도봉구 도봉동 신축급풀옵션 방큰원룸 역세권 도봉역 분 즉시입주 오픈형원룸 수...,서울시 도봉구 도봉동 신축급풀옵션 방큰원룸 역세권 도봉역 분 즉시입주 오픈형원룸 수...,"[서울시, 도봉구, 도봉동, 신축, 풀, 옵션, 방, 크다, 원룸, 역다, 권, 도...","[서울시 도봉구, 도봉구 도봉동, 도봉동 신축급풀옵션, 신축급풀옵션 방큰원룸, 방큰..."
39942,34522766,37.680244,127.044085,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔풀옵션_깨끗✔역세권_1호선도봉역 4분✔버스전용 20개노선 1. 교통편리 : 역세권...,https://ic.zigbang.com/ic/items/34522766/1.jpg,월세,300,30.0,3.0,서울시 도봉구 도봉동 풀옵션 깨끗 역세권 호선도봉역 분 버스전용 개노선 교통편리 역...,서울시 도봉구 도봉동 풀옵션 깨끗 역세권 호선도봉역 분 버스전용 개노선 교통편리 역...,"[서울시, 도봉구, 도봉동, 풀, 옵션, 깨끗, 역다, 권, 도봉역, 버스, 전용,...","[서울시 도봉구, 도봉구 도봉동, 도봉동 풀옵션, 풀옵션 깨끗, 깨끗 역세권, 역세..."
39943,34411514,37.680953,127.043451,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 도봉구 도봉동,🌈도봉역 2019년식 투룸 안심대출가능 주차가능🌈 📌매물정보\r\n 도봉역5...,https://ic.zigbang.com/ic/items/34411514/1.jpg,전세,20400,NaN,4.0,서울시 도봉구 도봉동 도봉역 년식 투룸 안심대출가능 주차가능 매물정보 도봉역 분 분...,서울시 도봉구 도봉동 도봉역 년식 투룸 안심대출가능 주차가능 매물정보 도봉역 분 분...,"[서울시, 도봉구, 도봉동, 도봉역, 투룸, 안심, 대출, 가능, 주차, 가능, 도...","[서울시 도봉구, 도봉구 도봉동, 도봉동 도봉역, 도봉역 년식, 년식 투룸, 투룸 ..."
39944,34575820,37.680461,127.043434,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.

### 유사매물 학습하기

In [42]:
result = data.token_new
model = ft(result, vector_size=100, window=5, min_count=5, workers=5, sg=1)

In [43]:
model.wv['주차가능']

array([-0.02610061, -0.05349438, -0.11829033, -0.09406196,  0.02102788,
        0.05125005, -0.04425063, -0.04450341, -0.06641188, -0.05386136,
        0.05999469,  0.00777189, -0.1041589 , -0.0101501 , -0.07038213,
       -0.07519357, -0.07100961,  0.12425134, -0.01403517,  0.00838643,
        0.03768564,  0.06637917, -0.05257117,  0.04664491, -0.1091194 ,
        0.00791844,  0.09922452,  0.11373997, -0.03127642, -0.07031946,
       -0.02607579,  0.01961439,  0.01990333, -0.04045841,  0.0192666 ,
       -0.07281043,  0.06252198, -0.08326277, -0.09556004, -0.1287621 ,
       -0.02496309, -0.16023278,  0.10500815, -0.03761712,  0.00805811,
       -0.06602778,  0.05976412, -0.02102473, -0.05072767, -0.01515324,
        0.08006714, -0.02739362, -0.11296152,  0.05437125, -0.01009908,
       -0.0115329 , -0.00901869,  0.00808771, -0.18776691, -0.10989761,
       -0.00111842, -0.04847123, -0.01893323,  0.06493375, -0.08241431,
        0.04393459, -0.04512918, -0.00771144, -0.01167111,  0.04

In [44]:
model.wv.most_similar('반려동물 가능한')

[('메이플', 0.35676780343055725),
 ('화곡역', 0.3485051393508911),
 ('초학', 0.3430761694908142),
 ('우장산역', 0.33924761414527893),
 ('발산역', 0.3389747440814972),
 ('두번째', 0.337208092212677),
 ('까치산역', 0.33293071389198303),
 ('응암역', 0.32837948203086853),
 ('내발산동', 0.3283321261405945),
 ('알짜', 0.32677122950553894)]

In [45]:
def sentence_vectorizer(input_sentence):
    imsi = None
    for i in input_sentence:
        if imsi is None:
            imsi = i
        else:
            imsi += i
    

    return imsi / len(input_sentence)

In [ ]:
sentence_vectors = []

for i in data['token_new']:
    sentence_vectors.append(sentence_vectorizer(model.wv[i]))

In [ ]:
data.drop(['vector_uni'], axis=1, inplace=True)

In [39]:
data['vector_uni'] = sentence_vectors

In [40]:
data

,id,lat,lng,info,url,address,detail,picture,type,deposit,rental_fee,maintenance_fee,doc,doc_new,token_new,bigram_list,vector_bi,vector_uni
144,34402581,37.446504,127.057650,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,"◆◆전입가,무융자,전세대출가,지하철역3분◆이중창,주차가능 ◆◆전입가,무융자,전세대출...",https://ic.zigbang.com/ic/items/34402581/1.jpg,전세,17500,NaN,8.0,서울시 서초구 신원동 전입가 무융자 전세대출가 지하철역 분 이중창 주차가능 전입가 ...,서울시 서초구 신원동 전입가 무융자 전세대출가 지하철역 분 이중창 주차가능 전입가 ...,"[서울시, 서초구, 신원동, 전입, 무, 융자, 대출, 지하철역, 이중창, 주차, ...","[서울시 서초구, 서초구 신원동, 신원동 전입가, 전입가 무융자, 무융자 전세대출가...","[-0.07083422, -0.041741773, -0.22743727, -0.01...","[-0.2796379, -0.26217353, -0.3682719, 0.003365..."
145,34448303,37.446532,127.057644,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34448303/1.jpg,월세,1000,65.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,"[서울시, 서초구, 신원동, 청계산, 입구, 역다, 권, 오피스텔, 청계산, 입구,...","[서울시 서초구, 서초구 신원동, 신원동 청계산입구, 청계산입구 초, 초 역세권, ...","[-0.2668152, -0.24815576, -0.14407004, -0.2503...","[-0.27112588, -0.50000226, -0.3416491, 0.00309..."
159,34521223,37.447032,127.056570,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 서초구 신원동,★신축급 풀옵션 투룸 오피스텔 전세 매물★ # 12월8일 이후부터 투어 가능\r\...,https://ic.zigbang.com/ic/items/34521223/1.jpg,전세,37500,NaN,20.0,서울시 서초구 신원동 신축급 풀옵션 투룸 오피스텔 전세 매물 월 일 이후부터 투어 ...,서울시 서초구 신원동 신축급 풀옵션 투룸 오피스텔 전세 매물 월 일 이후부터 투어 ...,"[서울시, 서초구, 신원동, 신축, 풀, 옵션, 투룸, 오피스텔, 전세, 투어, 가...","[서울시 서초구, 서초구 신원동, 신원동 신축급, 신축급 풀옵션, 풀옵션 투룸, 투...","[-0.1718661, -0.2121297, -0.103629045, 0.13494...","[-0.34929365, -0.3600866, -0.44582617, 0.20870..."
165,34493219,37.446730,127.056288,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34493219/1.jpg,전세,17000,NaN,8.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,"[서울시, 서초구, 신원동, 청계산, 입구, 역다, 권, 오피스텔, 청계산, 입구,...","[서울시 서초구, 서초구 신원동, 신원동 청계산입구, 청계산입구 초, 초 역세권, ...","[-0.2844597, -0.23775868, -0.13359916, -0.2426...","[-0.43222982, -0.5128427, -0.3245619, 0.023184..."
188,34498546,37.447738,127.056370,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 사업자용 *청계산입구 초 역세권 신축 오피스텔\r\n...,https://ic.zigbang.com/ic/items/34498546/1.jpg,월세,1000,88.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 사업자용 청계산입구 초 역세권 ...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 사업자용 청계산입구 초 역세권 ...,"[서울시, 서초구, 신원동, 청계산, 입구, 역다, 권, 오피스텔, 사업자, 용, ...","[서울시 서초구, 서초구 신원동, 신원동 청계산입구, 청계산입구 초, 초 역세권, ...","[-0.23046526, -0.24875703, -0.1277101, -0.2390...","[-0.16958399, -0.45018145, -0.31346875, 0.0042..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39941,34512135,37.680000,127.042272,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔신축급풀옵션_방큰원룸✔역세권_도봉역 4분✔즉시입주 1. 오픈형원룸/수려한 도봉산경...,https://ic.zigbang.com/ic/items/34512135/1.jpg,월세,500,37.0,3.0,서울시 도봉구 도봉동 신축급풀옵션 방큰원룸 역세권 도봉역 분 즉시입주 오픈형원룸 수...,서울시 도봉구 도봉동 신축급풀옵션 방큰원룸 역세권 도봉역 분 즉시입주 오픈형원룸 수...,"[서울시, 도봉구, 도봉동, 신축, 풀, 옵션, 방, 크다, 원룸, 역다, 권, 도...","[서울시 도봉구, 도봉구 도봉동, 도봉동 신축급풀옵션, 신축급풀옵션 방큰원룸, 방큰...","[-0.1638528, 0.22492363, 0.2472996, 0.10029973...","[-0.15263483, 0.2911351, 0.13714139, 0.0636072..."
39942,34522766,37.680244,127.044085,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔풀옵션_깨끗✔역세권_1호선도봉역 4분✔버스전용 20개노선 1. 교통편리 : 역세권...,https://ic.zigbang.com/ic/items/34522766/1.jpg,월세,300,30.0,3.0,서울시 도봉구 도봉동 풀옵션 깨끗 역세권 호선도봉역 분 버스전용 개노선 교통편리 역...,서울시 도봉구 도봉동 풀옵션 깨끗 역세권 호선도봉역 분 버스전용 개노선 교통편리 역...,"[서울시, 도봉구,

In [33]:
# 비슷한 매물 상위 10개 추천
input_data = input('찾고자 하는 매물의 정보를 입력해주세요:')

input_vector = model.wv[input_data]
cosine_similarities = cosine_similarity(sentence_vectors, [input_vector])
sim_scores = list(enumerate(cosine_similarities))
sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)

rec_index = [i[0] for i in sim_scores]
rec_index = rec_index[:10]

recommend = data.iloc[rec_index]
recommend

찾고자 하는 매물의 정보를 입력해주세요:애완동물 가능


,id,lat,lng,info,url,address,detail,picture,type,deposit,rental_fee,maintenance_fee,doc,doc_new,token_new,bigram_list,vector
16025,34319228,37.562980,127.085159,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/343...,서울시 광진구 중곡동,깔끔한 투룸월세. 전세도 가능합니다. * 보증금 25000만원 전세도 가능합니다.\...,https://ic.zigbang.com/ic/items/34319228/1.jpg,월세,21000,20.0,5.0,서울시 광진구 중곡동 깔끔한 투룸월세 전세도 가능합니다 보증금 만원 전세도 가능합니...,서울시 광진구 중곡동 깔끔한 투룸월세 전세도 가능 보증금 만원 전세도 가능 주차가능...,"[서울시, 광진구, 중곡동, 깔끔하다, 투룸, 월세, 전세, 가능, 보증금, 전세,...","[서울시 광진구, 광진구 중곡동, 중곡동 깔끔한, 깔끔한 투룸월세, 투룸월세 전세도...","[-0.6995179, 0.0137868505, -0.4260109, 0.06414..."
6158,34578527,37.499507,127.098159,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 송파구 석촌동,💜도배장판교체완료💜보증금조정가능💜주차가능💜 석촌고분역 도보거리\r\n\r\n방 두개...,https://ic.zigbang.com/ic/items/34578527/1.jpg,월세,5000,80.0,5.0,서울시 송파구 석촌동 도배장판교체완료 보증금조정가능 주차가능 석촌고분역 도보거리 방...,서울시 송파구 석촌동 도배장판교체완료 보증금조정가능 주차가능 석촌고분역 도보거리 방...,"[서울시, 송파구, 석촌동, 도배, 장판, 교체, 완료, 보증금, 조정, 가능, 주...","[서울시 송파구, 송파구 석촌동, 석촌동 도배장판교체완료, 도배장판교체완료 보증금조...","[-0.34639004, -0.039001845, -0.2676711, -0.078..."
3552,34436260,37.484216,127.129701,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 송파구 문정동,★ 말도안되는 가성비 애완가능 주차가능 쓰리룸 ★ \r\n★ 애완동물 가능\r\n\...,https://ic.zigbang.com/ic/items/34436260/1.jpg,전세,20000,NaN,0.0,서울시 송파구 문정동 말도안되는 가성비 애완가능 주차가능 쓰리룸 애완동물 가능 주차...,서울시 송파구 문정동 말도안되는 가성비 애완가능 주차가능 쓰리룸 애완동물 가능 주차...,"[서울시, 송파구, 문정동, 말, 안되다, 가성, 애완, 가능, 주차, 가능, 쓰리...","[서울시 송파구, 송파구 문정동, 문정동 말도안되는, 말도안되는 가성비, 가성비 애...","[-0.6872019, -0.15595096, -0.32132858, -0.0346..."
23461,34507284,37.610929,127.074669,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 중랑구 묵동,"LH가능,올수리된 쓰리룸,마당 사용가능 입주는 1월 초순 이후로 가능합니다.\r\n...",https://ic.zigbang.com/ic/items/34507284/1.jpg,전세,15000,NaN,0.0,서울시 중랑구 묵동 가능 올수리된 쓰리룸 마당 사용가능 입주는 월 초순 이후로 가능...,서울시 중랑구 묵동 가능 올수리된 쓰리룸 마당 사용가능 입주는 월 초순 이후로 가능...,"[서울시, 중랑구, 묵동, 가능, 수리, 쓰리룸, 마당, 사용, 가능, 초순, 가능...","[서울시 중랑구, 중랑구 묵동, 묵동 가능, 가능 올수리된, 올수리된 쓰리룸, 쓰리...","[-0.53553504, 0.0239081, -0.36524716, -0.10860..."
25112,34582250,37.458295,126.897063,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 금천구 시흥동,"🌈헬스장무료, 중기청가능, 주차가능, 빠른입주 가능 ! 🌈 중기청 대출 가능\r\n...",https://ic.zigbang.com/ic/items/34582250/1.jpg,전세,12500,NaN,10.0,서울시 금천구 시흥동 헬스장무료 중기청가능 주차가능 빠른입주 가능 중기청 대출 가능...,서울시 금천구 시흥동 헬스장무료 중기청가능 주차가능 빠른입주 가능 중기청 대출 가능...,"[서울시, 금천구, 시흥동, 헬스장, 무료, 중기청, 가능, 주차, 가능, 빠르다,...","[서울시 금천구, 금천구 시흥동, 시흥동 헬스장무료, 헬스장무료 중기청가능, 중기청...","[-0.6500284, -0.34156668, -0.13285792, -0.0286..."
8367,34489380,37.507730,127.087623,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 송파구 잠실동,🍀 올리모델링 역세권 원룸 🍀 🍀 아쉽게 주차는 안됩니다\r\n🍀 여자분 입주 선호...,https://ic.zigbang.com/ic/items/34489380/1.jpg,월세,1000,65.0,10.0,서울시 송파구 잠실동 올리모델링 역세권 원룸 아쉽게 주차는 안됩니다 여자분 입주 선...,서울시 송파구 잠실동 올리모델링 역세권 원룸 아쉽게 주차는 불가능 여자분 입주 선호...,"[서울시, 송파구, 잠실동, 올리다, 모델링, 역다, 권, 원룸, 아쉽다, 주차, ...","[서울시 송파구, 송파구 잠실동, 잠실동 올리모델링, 올리모델링 역세권, 역세권 원...","[-0.558497, -0.06698213, -0.29927808, 0.054549..."
22153,34511883,37.599488,127.076882,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 중랑구 중화동,"올수리,단독공간좋음,반려동물,즉시입주,전세대출,주차 ※중화동 다원부동산※\r\n\r...",https://ic.zigbang.com/ic/items/34511883/1.jpg,전세,25000,NaN,0.0,서울시 중랑구 중화동 올수리 단독공간좋음 반려동물 즉시입주 전세대출 주차 중화동 다...,서울시 중랑구 중화동 올수리 단독공간좋음 불가능동물 즉시입주 전세대출 주차 중화동 ...,"[서울시, 중랑구, 중화동, 수리, 독공, 좋다, 불가능, 동물, 전세, 대출, 주...","[서울시 중랑구, 중랑구 중화동, 중화동 올수리, 올수리 단독공간좋음, 단독공간좋음...","[-0.4182439, -0.002454624, -0.34879944, -0.165..."
39114,34454734,37.644041,127.036084,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 도봉구 창동,"수리완료,lh가능,주차가, ★lh 전세대출 가능 반전세 1억/25 가능성\...",https://ic.zigbang.com/ic/items/34454734/1.jpg,전세,

In [35]:
for i in recommend['doc']:
    print(i)

서울시 광진구 중곡동 깔끔한 투룸월세 전세도 가능합니다 보증금 만원 전세도 가능합니다 주차가능하며 전철역은 도보 분입니다 보증금대출 가능하나 반려동물은 안됩니다 날짜는 협의 가능합니다 
서울시 송파구 석촌동 도배장판교체완료 보증금조정가능 주차가능 석촌고분역 도보거리 방 두개 모두 커서 각방 사용 가능합니다 작은 방에 깊은 벽장 있어 계절용품이나 잠다한 물건 수납 가능합니다 통베란다 있어 세탁실 및 다양한 용도로 사옹 가능합니다 주방겸 거실이 넓어 도어 냉장고와 식탁배치 가능해요 위반 없는 매물로 전세대출 가능합니다 주차 대 가능합니다 애완동물 사육 불가 보증금 억까지 조정 가능합니다 는 불가합니다 
서울시 송파구 문정동 말도안되는 가성비 애완가능 주차가능 쓰리룸 애완동물 가능 주차 무료 대 가능 전세자금 대출 가능 문정역 역세권 
서울시 중랑구 묵동 가능 올수리된 쓰리룸 마당 사용가능 입주는 월 초순 이후로 가능합니다 반려동물 가능합니다 깊지않은 반지하이며 층과 반지층 두가구만 사는 단독주택입니다 올리모델링 상태 마당이 있어 전원주택 느낌으로 거주할 수 있습니다 주차는 불가합니다 보증금 억 천만 월세 으로 조정 가능합니다 전세 대출 가능합니다 
서울시 금천구 시흥동 헬스장무료 중기청가능 주차가능 빠른입주 가능 중기청 대출 가능 헬스장과 골프장 무료 시설로 이용 가능 남동향으로 채광 굳 빠른 입주 가능 주차 대 가능 
서울시 송파구 잠실동 올리모델링 역세권 원룸 아쉽게 주차는 안됩니다 여자분 입주 선호하십니다 보증금 조절 가능합니다 만 원 까지 가능 입주는 언제든 협조 가능합니다 학원 선생님 강력추천 잠실새내 직장이신분 강력추천 보증금 대출은 안됩니다 
서울시 중랑구 중화동 올수리 단독공간좋음 반려동물 즉시입주 전세대출 주차 중화동 다원부동산 올수리 단독주택 단독공간활용가능 즉시입주 가능합니다 공간활용하시기 정말 좋습니다 반려동물과 함께 생활이 가능합니다 공간좋은 투룸으로 넓게 사용가능해요 전세대출 가능하며 주차도 대 가능합니다 정남향 밝은집 
서울시 도봉구 창

In [31]:
with open('fasttext_model_1220.pickle', 'wb') as fw:
    pickle.dump(model,fw)

In [32]:
with open('seoulroom_list_1220.pickle', 'wb') as fw:
    pickle.dump(data,fw)